In [7]:
import multiprocessing as mp
mp.set_start_method("spawn", force=True)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import argparse
import json
import logging
import warnings
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image, ImageEnhance
from torch.utils.data import DataLoader, Dataset
from transformers import SamModel, SamProcessor
import shutil
from concurrent.futures import ProcessPoolExecutor, as_completed
warnings.filterwarnings('ignore')

# ===== DEVICE CONFIG =====
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    logging.info(f"GPU count visible to torch: {torch.cuda.device_count()}")
    logging.info(f"Using device: {device}")
    logging.info(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    logging.info("Using CPU")

# ===== IMPORT MODEL & AUGMENT =====
import albumentations as A
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2

# ===== CONFIG =====
CKPT_PATH  = "models/best_unet_mnv2.pth"  
IMG_PATH   = "/home/bbsw/thong/deep_learning/tk1/data/padtor_doctor/blight/100178.jpg"
IMG_SIZE   = 256
PROB_THRESH = 0.5

# ===== LOAD MODEL =====
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) Định nghĩa kiến trúc đúng như lúc train
model = smp.Unet(
    encoder_name="mobilenet_v2",
    encoder_weights=None,  # không dùng pretrained vì đã có checkpoint
    in_channels=3,
    classes=1
)

# 2) Load checkpoint
state = torch.load(CKPT_PATH, map_location=device)
if isinstance(state, dict) and any(k in state for k in ["state_dict", "model", "net"]):
    state = state.get("state_dict", state.get("model", state.get("net", state)))

from collections import OrderedDict
if next(iter(state.keys())).startswith("module."):
    new_state = OrderedDict((k.replace("module.", ""), v) for k, v in state.items())
    state = new_state

model.load_state_dict(state, strict=True)
model.to(device).eval().to(memory_format=torch.channels_last)

# ===== TIỀN XỬ LÝ ẢNH =====
tf = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ToTensorV2()
])

# ===== ĐỌC ẢNH =====
img_bgr = cv2.imread(IMG_PATH, cv2.IMREAD_COLOR)
H, W = img_bgr.shape[:2]
x = tf(image=cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB))["image"].unsqueeze(0)
x = x.to(device).to(memory_format=torch.channels_last)

# ===== INFERENCE =====
with torch.inference_mode(), torch.cuda.amp.autocast(enabled=(device.type == "cuda")):
    logit = model(x)
prob = torch.sigmoid(logit)[0, 0].detach().cpu().numpy()
mask_small = (prob > PROB_THRESH).astype(np.uint8)
mask = cv2.resize(mask_small, (W, H), interpolation=cv2.INTER_NEAREST)

# ===== LƯU MASK =====
output_dir = Path("segmentation_results")
output_dir.mkdir(parents=True, exist_ok=True)

out_path = output_dir / f"{Path(IMG_PATH).stem}_mask.png"
cv2.imwrite(str(out_path), (mask * 255))

print(f"✅ Mask saved to: {out_path}")


✅ Mask saved to: segmentation_results/100178_mask.png


In [4]:
!pip install albumentations
!pip install segmentation-models-pytorch

  Using cached segmentation_models_pytorch-0.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached timm-1.0.20-py3-none-any.whl.metadata (61 kB)
Using cached segmentation_models_pytorch-0.5.0-py3-none-any.whl (154 kB)
Using cached timm-1.0.20-py3-none-any.whl (2.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [segmentation-models-pytorch]on-models-pytorch]


In [2]:
import os
import cv2
import gc
import json
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

# ===== CONFIG =====
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   # ✅ chỉ dùng GPU thứ 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Using:", torch.cuda.get_device_name(0))

DATA_ROOT   = "/home/bbsw/thong/deep_learning/tk1/data/new_data_field_rice"
CKPT_PATH   = "/home/bbsw/truong/train/models/best_unet_mnv2.pth"
OUTPUT_ROOT = "/home/bbsw/thong/deep_learning/tk1/data/new_data_field_rice_detected/unet_gpu1_fast"
IMG_SIZE    = 256
PROB_THRESH = 0.5
MAX_WORKERS = 4   # số luồng song song (4 là tối ưu cho 1 GPU 4090)

# ===== LOAD MODEL 1 LẦN DUY NHẤT =====
model = smp.Unet(
    encoder_name="mobilenet_v2",
    encoder_weights=None,
    in_channels=3,
    classes=1
)
state = torch.load(CKPT_PATH, map_location=device)
if "state_dict" in state:
    state = state["state_dict"]
from collections import OrderedDict
if next(iter(state.keys())).startswith("module."):
    state = OrderedDict((k.replace("module.",""), v) for k,v in state.items())
model.load_state_dict(state, strict=True)
model.to(device).eval().to(memory_format=torch.channels_last)
print("✅ Model loaded.")

# ===== TRANSFORM =====
tf = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225)),
    ToTensorV2()
])

# ===== UTILS =====
def create_overlay(original, mask, color=(255,0,0), alpha=0.5):
    overlay = original.copy()
    cmask = np.zeros_like(original)
    cmask[mask>0] = color
    return cv2.addWeighted(overlay, 1-alpha, cmask, alpha, 0)

def extract_region(original, mask):
    if mask.max() == 0: return None
    coords = np.column_stack(np.where(mask > 0))
    y1, x1 = coords.min(axis=0)
    y2, x2 = coords.max(axis=0)
    pad = 10
    h, w = original.shape[:2]
    y1, x1 = max(0, y1-pad), max(0, x1-pad)
    y2, x2 = min(h, y2+pad), min(w, x2+pad)
    return original[y1:y2, x1:x2]

# ===== WORKER =====
def process_one(img_path: str, out_dir: Path):
    img = cv2.imread(img_path)
    H, W = img.shape[:2]
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = tf(image=rgb)["image"].unsqueeze(0).to(device)

    with torch.inference_mode(), torch.cuda.amp.autocast(enabled=True):
        logit = model(x)
    prob = torch.sigmoid(logit)[0,0].cpu().numpy()
    mask = (cv2.resize((prob > PROB_THRESH).astype(np.uint8), (W,H)) * 255).astype(np.uint8)

    stem = Path(img_path).stem
    out_dir.mkdir(parents=True, exist_ok=True)
    mask_path = out_dir / f"{stem}_mask.png"
    overlay_path = out_dir / f"{stem}_overlay.png"
    region_path = out_dir / f"{stem}_region.png"

    cv2.imwrite(str(mask_path), mask)
    over = create_overlay(rgb, mask)
    cv2.imwrite(str(overlay_path), cv2.cvtColor(over, cv2.COLOR_RGB2BGR))
    region = extract_region(rgb, mask)
    if region is not None:
        cv2.imwrite(str(region_path), cv2.cvtColor(region, cv2.COLOR_RGB2BGR))
    return {"image": img_path, "mask": str(mask_path)}

# ===== MAIN =====
def main():
    Path(OUTPUT_ROOT).mkdir(parents=True, exist_ok=True)
    disease_dirs = [p for p in Path(DATA_ROOT).iterdir() if p.is_dir()]
    print(f"🧩 Found {len(disease_dirs)} classes:", [d.name for d in disease_dirs])

    all_results = []
    for disease_dir in disease_dirs:
        out_dir = Path(OUTPUT_ROOT) / disease_dir.name
        imgs = [*disease_dir.glob("*.jpg"), *disease_dir.glob("*.png")]
        if not imgs: continue
        print(f"🔍 {disease_dir.name}: {len(imgs)} images")

        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
            futures = [ex.submit(process_one, str(img), out_dir) for img in imgs]
            for i, f in enumerate(as_completed(futures), 1):
                try:
                    res = f.result()
                    all_results.append(res)
                    if i % 10 == 0:
                        print(f"➡️ {i}/{len(imgs)} done")
                except Exception as e:
                    print(f"❌ {e}")
        torch.cuda.empty_cache(); gc.collect()

    with open(Path(OUTPUT_ROOT)/"summary.json", "w") as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)
    print(f"✅ Done! Results at {OUTPUT_ROOT}")

if __name__ == "__main__":
    main()


🚀 Using: NVIDIA GeForce RTX 4090
✅ Model loaded.
🧩 Found 4 classes: ['brown_spot', 'leaf_blast', 'leaf_blight', 'healthy']
🔍 brown_spot: 965 images


/tmp/ipykernel_575864/4048015718.py:75: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.inference_mode(), torch.cuda.amp.autocast(enabled=True):


➡️ 10/965 done
➡️ 20/965 done
➡️ 30/965 done
➡️ 40/965 done
➡️ 50/965 done
➡️ 60/965 done
➡️ 70/965 done
➡️ 80/965 done
➡️ 90/965 done
➡️ 100/965 done
➡️ 110/965 done
➡️ 120/965 done
➡️ 130/965 done
➡️ 140/965 done
➡️ 150/965 done
➡️ 160/965 done
➡️ 170/965 done
➡️ 180/965 done
➡️ 190/965 done
➡️ 200/965 done
➡️ 210/965 done
➡️ 220/965 done
➡️ 230/965 done
➡️ 240/965 done
➡️ 250/965 done
➡️ 260/965 done
➡️ 270/965 done
➡️ 280/965 done
➡️ 290/965 done
➡️ 300/965 done
➡️ 310/965 done
➡️ 320/965 done
➡️ 330/965 done
➡️ 340/965 done
➡️ 350/965 done
➡️ 360/965 done
➡️ 370/965 done
➡️ 380/965 done
➡️ 390/965 done
➡️ 400/965 done
➡️ 410/965 done
➡️ 420/965 done
➡️ 430/965 done
➡️ 440/965 done
➡️ 450/965 done
➡️ 460/965 done
➡️ 470/965 done
➡️ 480/965 done
➡️ 490/965 done
➡️ 500/965 done
➡️ 510/965 done
➡️ 520/965 done
➡️ 530/965 done
➡️ 540/965 done
➡️ 550/965 done
➡️ 560/965 done
➡️ 570/965 done
➡️ 580/965 done
➡️ 590/965 done
➡️ 600/965 done
➡️ 610/965 done
➡️ 620/965 done
➡️ 630/965 done
➡

In [3]:
import os
import cv2
import gc
import json
import torch
import numpy as np
from ultralytics import YOLO
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

# ===== CONFIG =====
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   # ✅ chỉ dùng GPU thứ 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🚀 Using:", torch.cuda.get_device_name(0))

DATA_ROOT   = "/home/bbsw/thong/deep_learning/tk1/data/new_data_field_rice"
CKPT_PATH   = "/home/bbsw/thong/deep_learning/tk1/data/runs_rice_leaf/yolov8n_leaf_disease_v2/weights/epoch40.pt"
OUTPUT_ROOT = "/home/bbsw/thong/deep_learning/tk1/data/new_data_field_rice_detected/yolo_gpu1_fast"
CONF_THRESH = 0.4
IOU_THRESH  = 0.5
MAX_WORKERS = 4  # số luồng song song

# ===== LOAD YOLO MODEL 1 LẦN DUY NHẤT =====
print("📦 Loading YOLOv8 model...")
model = YOLO(CKPT_PATH)
model.to(device)
print("✅ Model loaded on", device)

# ===== UTILS =====
def draw_boxes(image, boxes, names, color=(0,255,0)):
    """Vẽ bounding boxes lên ảnh"""
    img = image.copy()
    for *xyxy, conf, cls in boxes:
        label = names[int(cls)]
        x1, y1, x2, y2 = map(int, xyxy)
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, f"{label} {conf:.2f}", (x1, y1 - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    return img

def save_crop(image, box, out_dir, stem, label):
    """Cắt và lưu vùng phát hiện"""
    x1, y1, x2, y2 = map(int, box[:4])
    crop = image[y1:y2, x1:x2]
    crop_path = Path(out_dir) / f"{stem}_{label}.png"
    cv2.imwrite(str(crop_path), crop)
    return str(crop_path)

# ===== WORKER =====
def process_one(img_path: str, out_dir: Path):
    img = cv2.imread(img_path)
    stem = Path(img_path).stem
    out_dir.mkdir(parents=True, exist_ok=True)

    results = model.predict(
        source=img,
        conf=CONF_THRESH,
        iou=IOU_THRESH,
        imgsz=640,
        device=device.index if device.type == "cuda" else "cpu",
        verbose=False
    )

    res = results[0]
    boxes = res.boxes.data.cpu().numpy() if len(res.boxes) else []
    names = model.names
    overlay = draw_boxes(img, boxes, names)

    overlay_path = out_dir / f"{stem}_overlay.jpg"
    cv2.imwrite(str(overlay_path), overlay)

    crops = []
    for box in boxes:
        label = names[int(box[5])]
        crop_path = save_crop(img, box, out_dir, stem, label)
        crops.append({"label": label, "path": crop_path})

    return {
        "image": img_path,
        "detections": len(boxes),
        "overlay": str(overlay_path),
        "crops": crops
    }

# ===== MAIN =====
def main():
    Path(OUTPUT_ROOT).mkdir(parents=True, exist_ok=True)
    disease_dirs = [p for p in Path(DATA_ROOT).iterdir() if p.is_dir()]
    print(f"🧩 Found {len(disease_dirs)} classes:", [d.name for d in disease_dirs])

    all_results = []
    for disease_dir in disease_dirs:
        out_dir = Path(OUTPUT_ROOT) / disease_dir.name
        imgs = [*disease_dir.glob("*.jpg"), *disease_dir.glob("*.png")]
        if not imgs:
            print(f"⚠️ No images in {disease_dir.name}")
            continue

        print(f"🔍 {disease_dir.name}: {len(imgs)} images")

        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as ex:
            futures = [ex.submit(process_one, str(img), out_dir) for img in imgs]
            for i, f in enumerate(as_completed(futures), 1):
                try:
                    res = f.result()
                    all_results.append(res)
                    if i % 10 == 0:
                        print(f"➡️ {i}/{len(imgs)} done for {disease_dir.name}")
                except Exception as e:
                    print(f"❌ Error: {e}")
        torch.cuda.empty_cache(); gc.collect()

    summary_path = Path(OUTPUT_ROOT) / "summary.json"
    with open(summary_path, "w") as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)
    print(f"\n✅ Done! Results at {OUTPUT_ROOT}")
    print(f"📄 Summary file: {summary_path}")

if __name__ == "__main__":
    main()


🚀 Using: NVIDIA GeForce RTX 4090
📦 Loading YOLOv8 model...
✅ Model loaded on cuda
🧩 Found 4 classes: ['brown_spot', 'leaf_blast', 'leaf_blight', 'healthy']
🔍 brown_spot: 965 images
❌ Error: bn
❌ Error: bn
❌ Error: bn
Ultralytics 8.3.204 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 22684MiB)
Ultralytics 8.3.204 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 22684MiB)
Ultralytics 8.3.204 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 22684MiB)
Ultralytics 8.3.204 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 22684MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs
➡️ 10/965 done for brown_spot
➡️ 20/965 done for brown_spot
➡️ 30/965 d

KeyboardInterrupt: 

In [ ]:
import os
import cv2
import gc
import json
import math
import torch
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from typing import List

# ============ CONFIG ============
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"      # chỉ expose GPU index=1 ra môi trường

DATA_ROOT    = "/home/bbsw/thong/deep_learning/tk1/data/new_data_field_rice"
CKPT_PATH    = "/home/bbsw/thong/deep_learning/tk1/data/runs_rice_leaf/yolov8n_leaf_disease_v2/weights/best.pt"
OUTPUT_ROOT  = "/home/bbsw/thong/deep_learning/tk1/data/new_data_field_rice_detected/yolo_gpu1_fast"
CONF_THRESH  = 0.4
IOU_THRESH   = 0.5
IMG_SIZE     = 640
BATCH_SIZE   = 16            # tăng/giảm tùy VRAM
MAX_DET      = 300
OVERLAY_EVERY = 1            # =1 lưu overlay mọi ảnh; tăng lên nếu muốn giảm I/O
ERROR_LOG    = "errors.jsonl"

# ============ DEVICE ============
use_cuda = torch.cuda.is_available()
if use_cuda:
    # Chú ý: sau khi set CUDA_VISIBLE_DEVICES="1", GPU hiển thị sẽ là index 0 đối với tiến trình này
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🚀 Using CUDA: {gpu_name}")
    yolo_device = 0
else:
    print("🚀 Using CPU")
    yolo_device = "cpu"

# ============ HELPERS ============
VALID_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

def is_image_file(p: Path) -> bool:
    return p.suffix.lower() in VALID_EXTS

def sanitize_filename(text: str) -> str:
    # loại bỏ ký tự có thể gây lỗi filesystem
    bad = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for b in bad:
        text = text.replace(b, '_')
    return text.strip() or "unk"

def draw_boxes(image_bgr: np.ndarray, boxes: np.ndarray, names: dict) -> np.ndarray:
    img = image_bgr.copy()
    for *xyxy, conf, cls in boxes:
        label = names.get(int(cls), f"id{int(cls)}")
        x1, y1, x2, y2 = map(int, xyxy)
        x1 = max(0, min(x1, img.shape[1]-1))
        x2 = max(0, min(x2, img.shape[1]-1))
        y1 = max(0, min(y1, img.shape[0]-1))
        y2 = max(0, min(y2, img.shape[0]-1))
        if x2 <= x1 or y2 <= y1:
            continue
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
        txt = f"{label} {float(conf):.2f}"
        cv2.putText(img, txt, (x1, max(0, y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
    return img

def save_crop(image_bgr: np.ndarray, box: np.ndarray, out_dir: Path, stem: str, label: str) -> str:
    x1, y1, x2, y2 = map(int, box[:4])
    H, W = image_bgr.shape[:2]
    x1 = max(0, min(x1, W-1)); x2 = max(0, min(x2, W-1))
    y1 = max(0, min(y1, H-1)); y2 = max(0, min(y2, H-1))
    if x2 <= x1 or y2 <= y1:
        return ""
    crop = image_bgr[y1:y2, x1:x2]
    if crop.size == 0:
        return ""
    stem_s = sanitize_filename(stem)
    label_s = sanitize_filename(label)
    crop_path = out_dir / f"{stem_s}_{label_s}.png"
    ok = cv2.imwrite(str(crop_path), crop)
    return str(crop_path) if ok else ""

def chunk_list(items: List[Path], n: int) -> List[List[Path]]:
    return [items[i:i+n] for i in range(0, len(items), n)]

def write_err(err_path: Path, payload: dict):
    with open(err_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

# ============ LOAD MODEL ============
print("📦 Loading YOLOv8 model...")
model = YOLO(CKPT_PATH)
model.to(yolo_device)
names = model.names
print(f"✅ Model loaded on device={yolo_device}")

# ============ MAIN ============
def main():
    out_root = Path(OUTPUT_ROOT)
    out_root.mkdir(parents=True, exist_ok=True)
    err_file = out_root / ERROR_LOG
    if err_file.exists():
        err_file.unlink()  # clear old

    disease_dirs = [p for p in Path(DATA_ROOT).iterdir() if p.is_dir()]
    print(f"🧩 Found {len(disease_dirs)} classes:", [d.name for d in disease_dirs])

    all_results = []

    for disease_dir in disease_dirs:
        imgs = [p for p in disease_dir.iterdir() if p.is_file() and is_image_file(p)]
        if not imgs:
            print(f"⚠️ No images in {disease_dir.name}")
            continue

        print(f"🔍 {disease_dir.name}: {len(imgs)} images")
        out_dir = out_root / disease_dir.name
        out_dir.mkdir(parents=True, exist_ok=True)

        # Predict theo lô để ổn định GPU (không dùng multi-thread)
        batches = chunk_list(imgs, BATCH_SIZE)
        done = 0
        for bi, batch_paths in enumerate(batches, 1):
            # Ultralytics hỗ trợ truyền list đường dẫn trực tiếp
            try:
                results = model.predict(
                    source=[str(p) for p in batch_paths],
                    conf=CONF_THRESH,
                    iou=IOU_THRESH,
                    imgsz=IMG_SIZE,
                    device=yolo_device,
                    max_det=MAX_DET,
                    verbose=False,
                    batch=BATCH_SIZE
                )
            except Exception as e:
                # Log lỗi lô, sau đó xử từng ảnh để bỏ qua ảnh hỏng
                write_err(err_file, {"stage": "batch_predict", "class": disease_dir.name, "error": str(e)})
                # fallback: xử lý từng ảnh để xác định ảnh lỗi
                for p in batch_paths:
                    try:
                        _ = model.predict(
                            source=str(p),
                            conf=CONF_THRESH,
                            iou=IOU_THRESH,
                            imgsz=IMG_SIZE,
                            device=yolo_device,
                            max_det=MAX_DET,
                            verbose=False
                        )
                    except Exception as e1:
                        write_err(err_file, {"stage": "single_predict", "class": disease_dir.name, "image": str(p), "error": str(e1)})
                # tiếp tục batch tiếp theo
                continue

            # Hậu xử lý từng kết quả trong batch
            for p, res in zip(batch_paths, results):
                try:
                    img = cv2.imread(str(p), cv2.IMREAD_COLOR)
                    if img is None:
                        raise ValueError("cv2.imread returned None (unreadable/corrupted file)")
                    stem = p.stem
                    det = res.boxes
                    boxes = det.data.cpu().numpy() if det is not None and len(det) else np.empty((0, 6), dtype=float)

                    # Lưu overlay tùy chính sách
                    if OVERLAY_EVERY and (done % OVERLAY_EVERY == 0):
                        overlay = draw_boxes(img, boxes, names)
                        ov_path = out_dir / f"{sanitize_filename(stem)}_overlay.jpg"
                        cv2.imwrite(str(ov_path))
                        overlay_path = str(ov_path)
                    else:
                        overlay_path = ""

                    # Lưu crop theo từng box
                    crops = []
                    for box in boxes:
                        cls_id = int(box[5]) if len(box) >= 6 else 0
                        label = names.get(cls_id, f"id{cls_id}")
                        cpath = save_crop(img, box, out_dir, stem, label)
                        if cpath:
                            crops.append({"label": label, "path": cpath})

                    all_results.append({
                        "image": str(p),
                        "class_dir": disease_dir.name,
                        "detections": int(len(boxes)),
                        "overlay": overlay_path,
                        "crops": crops
                    })

                except Exception as eimg:
                    write_err(err_file, {"stage": "postprocess", "class": disease_dir.name, "image": str(p), "error": str(eimg)})

                done += 1
                if done % 10 == 0:
                    print(f"➡️ {done}/{len(imgs)} done for {disease_dir.name}")

            # Thu dọn VRAM sau mỗi batch
            del results
            torch.cuda.empty_cache() if use_cuda else None
            gc.collect()

    # Ghi tổng kết
    summary_path = Path(OUTPUT_ROOT) / "summary.json"
    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)

    print(f"\n✅ Done! Results at {OUTPUT_ROOT}")
    print(f"📄 Summary file: {summary_path}")
    if (Path(OUTPUT_ROOT)/ERROR_LOG).exists():
        print(f"⚠️ Error log: {Path(OUTPUT_ROOT)/ERROR_LOG}")

if __name__ == "__main__":
    main()


🚀 Using CUDA: NVIDIA GeForce RTX 4090
📦 Loading YOLOv8 model...
✅ Model loaded on device=0
🧩 Found 4 classes: ['brown_spot', 'leaf_blast', 'leaf_blight', 'healthy']
🔍 brown_spot: 965 images
➡️ 10/965 done for brown_spot
➡️ 20/965 done for brown_spot
➡️ 30/965 done for brown_spot
➡️ 40/965 done for brown_spot
➡️ 50/965 done for brown_spot
➡️ 60/965 done for brown_spot
➡️ 70/965 done for brown_spot
➡️ 80/965 done for brown_spot
➡️ 90/965 done for brown_spot
➡️ 100/965 done for brown_spot
➡️ 110/965 done for brown_spot
➡️ 120/965 done for brown_spot
➡️ 130/965 done for brown_spot
➡️ 140/965 done for brown_spot
➡️ 150/965 done for brown_spot
➡️ 160/965 done for brown_spot
➡️ 170/965 done for brown_spot
➡️ 180/965 done for brown_spot
➡️ 190/965 done for brown_spot
➡️ 200/965 done for brown_spot
➡️ 210/965 done for brown_spot
➡️ 220/965 done for brown_spot
➡️ 230/965 done for brown_spot
➡️ 240/965 done for brown_spot
➡️ 250/965 done for brown_spot
➡️ 260/965 done for brown_spot
➡️ 270/965 do

In [1]:
import os
import cv2
import gc
import json
import torch
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from typing import List

# ============ CONFIG ============
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"      # chỉ expose GPU index=1 ra môi trường

DATA_ROOT    = "/home/bbsw/thong/deep_learning/tk1/data/new_data_field_rice"
CKPT_PATH    = "/home/bbsw/thong/deep_learning/tk1/data/runs_rice_leaf/yolov8n_leaf_disease_v2/weights/best.pt"
OUTPUT_ROOT  = "/home/bbsw/thong/deep_learning/tk1/data/new_data_field_rice_detected/yolo_gpu1_fast"
CONF_THRESH  = 0.4
IOU_THRESH   = 0.5
IMG_SIZE     = 640
BATCH_SIZE   = 16
MAX_DET      = 300
OVERLAY_EVERY = 1            # =1: lưu overlay mọi ảnh
ERROR_LOG    = "errors.jsonl"

# ============ DEVICE ============
use_cuda = torch.cuda.is_available()
if use_cuda:
    gpu_name = torch.cuda.get_device_name(0)  # sau khi set CUDA_VISIBLE_DEVICES="1" → chỉ còn 1 GPU (index 0)
    print(f"🚀 Using CUDA: {gpu_name}")
    yolo_device = 0
else:
    print("🚀 Using CPU")
    yolo_device = "cpu"

# ============ HELPERS ============
VALID_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

def is_image_file(p: Path) -> bool:
    return p.suffix.lower() in VALID_EXTS

def sanitize_text(text: str) -> str:
    bad = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for b in bad:
        text = text.replace(b, '_')
    return text.strip() or "unk"

def save_image(path: Path, img: np.ndarray, err_file: Path, stage: str) -> bool:
    path.parent.mkdir(parents=True, exist_ok=True)
    img = np.ascontiguousarray(img)
    ok = False
    try:
        ok = cv2.imwrite(str(path), img)
    except Exception as e:
        with open(err_file, "a", encoding="utf-8") as f:
            f.write(json.dumps({"stage": stage, "path": str(path), "error": str(e)}, ensure_ascii=False) + "\n")
        return False
    if not ok:
        with open(err_file, "a", encoding="utf-8") as f:
            f.write(json.dumps({"stage": stage, "path": str(path), "error": "cv2.imwrite returned False"}, ensure_ascii=False) + "\n")
    return ok

def draw_boxes(image_bgr: np.ndarray, boxes: np.ndarray, names: dict) -> np.ndarray:
    img = image_bgr.copy()
    H, W = img.shape[:2]
    for *xyxy, conf, cls in boxes:
        label = names.get(int(cls), f"id{int(cls)}")
        x1, y1, x2, y2 = map(int, xyxy)
        x1 = max(0, min(x1, W-1)); x2 = max(0, min(x2, W-1))
        y1 = max(0, min(y1, H-1)); y2 = max(0, min(y2, H-1))
        if x2 <= x1 or y2 <= y1:
            continue
        cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
        txt = f"{label} {float(conf):.2f}"
        cv2.putText(img, txt, (x1, max(0, y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
    return img

def save_crop(image_bgr: np.ndarray, box: np.ndarray, out_dir: Path,
              stem: str, label: str, idx: int, conf: float, err_file: Path) -> str:
    H, W = image_bgr.shape[:2]
    x1, y1, x2, y2 = map(int, box[:4])
    x1 = max(0, min(x1, W-1)); x2 = max(0, min(x2, W-1))
    y1 = max(0, min(y1, H-1)); y2 = max(0, min(y2, H-1))
    if x2 <= x1 or y2 <= y1:
        return ""
    crop = image_bgr[y1:y2, x1:x2]
    if crop.size == 0:
        return ""
    stem_s  = sanitize_text(stem)
    label_s = sanitize_text(label)
    conf_s  = f"{conf:.2f}".replace('.', '')
    # thêm idx + conf để tránh ghi đè
    crop_path = out_dir / f"{stem_s}_{label_s}_{idx:02d}_{conf_s}.png"
    ok = save_image(crop_path, crop, err_file, stage="save_crop")
    return str(crop_path) if ok else ""

def chunk_list(items: List[Path], n: int) -> List[List[Path]]:
    return [items[i:i+n] for i in range(0, len(items), n)]

def write_err(err_path: Path, payload: dict):
    with open(err_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(payload, ensure_ascii=False) + "\n")

# ============ LOAD MODEL ============
print("📦 Loading YOLOv8 model...")
model = YOLO(CKPT_PATH)
model.to(yolo_device)
names = model.names
print(f"✅ Model loaded on device={yolo_device}")

# ============ MAIN ============
def main():
    out_root = Path(OUTPUT_ROOT)
    out_root.mkdir(parents=True, exist_ok=True)
    err_file = out_root / ERROR_LOG
    if err_file.exists():
        err_file.unlink()  # clear old

    disease_dirs = [p for p in Path(DATA_ROOT).iterdir() if p.is_dir()]
    print(f"🧩 Found {len(disease_dirs)} classes:", [d.name for d in disease_dirs])

    all_results = []

    for disease_dir in disease_dirs:
        imgs = [p for p in disease_dir.iterdir() if p.is_file() and is_image_file(p)]
        if not imgs:
            print(f"⚠️ No images in {disease_dir.name}")
            continue

        print(f"🔍 {disease_dir.name}: {len(imgs)} images")
        out_dir = out_root / disease_dir.name
        out_dir.mkdir(parents=True, exist_ok=True)

        batches = chunk_list(imgs, BATCH_SIZE)
        done = 0
        for bi, batch_paths in enumerate(batches, 1):
            try:
                results = model.predict(
                    source=[str(p) for p in batch_paths],
                    conf=CONF_THRESH,
                    iou=IOU_THRESH,
                    imgsz=IMG_SIZE,
                    device=yolo_device,
                    max_det=MAX_DET,
                    verbose=False,
                    batch=BATCH_SIZE
                )
            except Exception as e:
                write_err(err_file, {"stage": "batch_predict", "class": disease_dir.name, "error": str(e)})
                # fallback từng ảnh để ghi log file lỗi cụ thể
                for p in batch_paths:
                    try:
                        _ = model.predict(
                            source=str(p),
                            conf=CONF_THRESH,
                            iou=IOU_THRESH,
                            imgsz=IMG_SIZE,
                            device=yolo_device,
                            max_det=MAX_DET,
                            verbose=False
                        )
                    except Exception as e1:
                        write_err(err_file, {"stage": "single_predict", "class": disease_dir.name, "image": str(p), "error": str(e1)})
                continue

            # Hậu xử lý từng kết quả trong batch
            for p, res in zip(batch_paths, results):
                try:
                    img = cv2.imread(str(p), cv2.IMREAD_COLOR)
                    if img is None:
                        raise ValueError("cv2.imread returned None (unreadable/corrupted file)")
                    stem = p.stem
                    det = res.boxes
                    boxes = det.data.cpu().numpy() if det is not None and len(det) else np.empty((0, 6), dtype=float)

                    # Overlay
                    overlay_path = ""
                    if OVERLAY_EVERY and (done % OVERLAY_EVERY == 0):
                        overlay = draw_boxes(img, boxes, names)
                        ov_path = out_dir / f"{sanitize_text(stem)}_overlay.jpg"
                        save_image(ov_path, overlay, err_file, stage="save_overlay")
                        overlay_path = str(ov_path)

                    # Lưu từng crop
                    crops = []
                    for idx, box in enumerate(boxes, start=1):
                        cls_id = int(box[5]) if len(box) >= 6 else 0
                        label = names.get(cls_id, f"id{cls_id}")
                        conf  = float(box[4]) if len(box) >= 5 else 0.0
                        cpath = save_crop(img, box, out_dir, stem, label, idx, conf, err_file)
                        if cpath:
                            crops.append({"label": label, "conf": conf, "path": cpath})

                    all_results.append({
                        "image": str(p),
                        "class_dir": disease_dir.name,
                        "detections": int(len(boxes)),
                        "overlay": overlay_path,
                        "crops": crops
                    })

                except Exception as eimg:
                    write_err(err_file, {"stage": "postprocess", "class": disease_dir.name, "image": str(p), "error": str(eimg)})

                done += 1
                if done % 10 == 0:
                    print(f"➡️ {done}/{len(imgs)} done for {disease_dir.name}")

            # Thu dọn VRAM sau mỗi batch
            del results
            if use_cuda:
                torch.cuda.empty_cache()
            gc.collect()

    # Ghi tổng kết
    summary_path = Path(OUTPUT_ROOT) / "summary.json"
    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)

    print(f"\n✅ Done! Results at {OUTPUT_ROOT}")
    print(f"📄 Summary file: {summary_path}")
    if (Path(OUTPUT_ROOT)/ERROR_LOG).exists():
        print(f"⚠️ Error log: {Path(OUTPUT_ROOT)/ERROR_LOG}")

if __name__ == "__main__":
    main()


🚀 Using CUDA: NVIDIA GeForce RTX 4090
📦 Loading YOLOv8 model...
✅ Model loaded on device=0
🧩 Found 4 classes: ['brown_spot', 'leaf_blast', 'leaf_blight', 'healthy']
🔍 brown_spot: 965 images
➡️ 10/965 done for brown_spot
➡️ 20/965 done for brown_spot
➡️ 30/965 done for brown_spot
➡️ 40/965 done for brown_spot
➡️ 50/965 done for brown_spot
➡️ 60/965 done for brown_spot
➡️ 70/965 done for brown_spot
➡️ 80/965 done for brown_spot
➡️ 90/965 done for brown_spot
➡️ 100/965 done for brown_spot
➡️ 110/965 done for brown_spot
➡️ 120/965 done for brown_spot
➡️ 130/965 done for brown_spot
➡️ 140/965 done for brown_spot
➡️ 150/965 done for brown_spot
➡️ 160/965 done for brown_spot
➡️ 170/965 done for brown_spot
➡️ 180/965 done for brown_spot
➡️ 190/965 done for brown_spot
➡️ 200/965 done for brown_spot
➡️ 210/965 done for brown_spot
➡️ 220/965 done for brown_spot
➡️ 230/965 done for brown_spot
➡️ 240/965 done for brown_spot
➡️ 250/965 done for brown_spot
➡️ 260/965 done for brown_spot
➡️ 270/965 do